**Section 1: Import Libraries**

In [11]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [12]:
API_KEY = 'bae9f18460d0d5e4e2f3ed42d4b16e49'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'

**1. Fetch Current Weather Data**

In [13]:
def get_current_weather(city) :
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city' : data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like' : round(data['main']['feels_like']),
        'temp_min' : round(data['main']['temp_min']),
        'temp_max' : round(data['main']['temp_max']),
        'humidity' : round(data['main']['humidity']),
        'description' : data['weather'][0]['description'],
        'country' : data['sys']['country'],
        'wind_gust_dir': data['wind']['deg'],
        'pressure': data['main']['pressure'],
        'wind_gust_speed' : data['wind']['speed']
    }

**2. Read Historical Data**

In [14]:
def read_historical_data(filename) :
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

**3. Prepare data for training**

In [15]:
def prepare_data(data):
    le = LabelEncoder()
    data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
    data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

    #Definir les features et les targets
    X = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']] #feature
    y = data['RainTomorrow'] #target

    return X,y,le

**4. Train Rain Prediction Model**

In [16]:
def train_rain_model(X,y) :
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    print("Mean Squared Error For Rain Model")
    print(mean_squared_error(y_test,y_pred))

    return model

**5. Prepare regression data**

In [17]:
def prepare_regression_data(data, feature):
    X,y = [], []
    for i in range (len(data)-1) :
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])

    X = np.array(X).reshape(-1,1)
    y = np.array(y)
    return X,y

**Train Regression Model**

In [18]:
def train_regression_model(X,y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X,y)
    return model

**Predict Future**

In [19]:
def predict_future(model, current_value) :
    predictions = [current_value]
    for i in range(24) :
        next_value = model.predict(np.array([[predictions[-1]]]))

        predictions.append(next_value[0])

    return predictions[1:]

**Weather Analysis Function**

In [20]:
def weather_view():
    city = input("Enter any city name : ")
    current_weather = get_current_weather(city)

    #load historical data
    historical_data = read_historical_data("weather.csv")

    #prepare and train the rain prediction model

    X, y ,le = prepare_data(historical_data)

    rain_model = train_rain_model(X,y)

    #map wind direction to compass points

    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
        ("N", 0, 11.25), ("NNE",11.25,33.75), ("NE",33.75,56.25),
        ("ENE",56.25,78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
    ]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

    compass_direction_encoded = le.transform([compass_direction]) [0] if compass_direction in le.classes_ else -1

    current_data = {
        'MinTemp' : current_weather['temp_min'],
        'MaxTemp' : current_weather['temp_max'],
        'WindGustDir' : compass_direction_encoded,
        'WindGustSpeed' : current_weather['wind_gust_speed'],
        'Humidity' : current_weather['humidity'],
        'Pressure' : current_weather['pressure'],
        'Temp' : current_weather['current_temp']
    }

    current_df = pd.DataFrame([current_data])

    #rain prediction
    rain_prediction = rain_model.predict(current_df)[0]

    #prepare regrission model for temperature and humidity

    X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
    X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

    temp_model = train_regression_model(X_temp,y_temp)
    hum_model = train_regression_model(X_hum,y_hum)

    #predict future temperature and humidity

    future_temp = predict_future(temp_model,current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    #prepare time for future predictions

    timezone =pytz.timezone('Africa/Casablanca')
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0,second=0,microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(24)]

    #display results

    print(f"City : {city}, {current_weather['country']}")
    print(f"Current Temperature : {current_weather['current_temp']}")
    print(f"Feels Like: {current_weather['feels_like']}")
    print(f"Minimum Temperature: {current_weather['temp_min']}°C")
    print(f"Maximum Temperature: {current_weather['temp_max']}°C")
    print(f"Humidity: {current_weather['humidity']}%")
    print(f"Weather Prediction: {current_weather['description']}")
    print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

    print("\n Future Temperature Predictions")

    for time,temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")

    print("\n Future Humidity Predictions")

    for time,humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()

Mean Squared Error For Rain Model
0.1506849315068493
City : Marrakesh, MA
Current Temperature : 21
Feels Like: 20
Minimum Temperature: 21°C
Maximum Temperature: 21°C
Humidity: 40%
Weather Prediction: clear sky
Rain Prediction: No

 Future Temperature Predictions
18:00: 18.7°C
19:00: 20.2°C
20:00: 16.2°C
21:00: 15.4°C
22:00: 15.6°C
23:00: 18.2°C
00:00: 16.8°C
01:00: 21.4°C
02:00: 22.7°C
03:00: 25.8°C
04:00: 22.3°C
05:00: 23.4°C
06:00: 23.7°C
07:00: 22.9°C
08:00: 23.2°C
09:00: 18.0°C
10:00: 20.6°C
11:00: 21.3°C
12:00: 20.5°C
13:00: 22.2°C
14:00: 22.7°C
15:00: 25.0°C
16:00: 26.7°C
17:00: 24.4°C

 Future Humidity Predictions
18:00: 45.9%
19:00: 48.2%
20:00: 53.7%
21:00: 58.1%
22:00: 54.2%
23:00: 58.1%
00:00: 54.2%
01:00: 58.1%
02:00: 54.2%
03:00: 58.1%
04:00: 54.2%
05:00: 58.1%
06:00: 54.2%
07:00: 58.1%
08:00: 54.2%
09:00: 58.1%
10:00: 54.2%
11:00: 58.1%
12:00: 54.2%
13:00: 58.1%
14:00: 54.2%
15:00: 58.1%
16:00: 54.2%
17:00: 58.1%
